In [41]:
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import urlopen
import time
import numpy as np
from pandas import DataFrame
import shutil

In [42]:
'''
Get response:
 => Not as robust as it should be
 => If not working, try change the my_headers[] used in the 24th line to a different header
     
'''
def get_response(url):
    my_headers = [
     'Mozilla/5.0',
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",     "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
]
    req = request.Request(url)
#     req.add_header('User-Agent', random.choice(my_headers))
    req.add_header('User-Agent',my_headers[3])
    response = urlopen(req)
    page = response.read()
#     if response.getcode()==200:
#         page = response.read()
#     else:
#         raise Exception('Can\'t get page ')
    return page

In [46]:
'''
Get images on a specific image address:
 => Need to change file directory as needed
'''
def get_images(url):
        # Open the url image, set stream to True, this will return the stream content.
        resp = requests.get(url, stream=True)
        # Open a local file with wb ( write binary ) permission.
        local_file = open('./images/local_image.jpg', 'wb')
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        resp.raw.decode_content = True
        # Copy the response stream raw data to local image file.
        shutil.copyfileobj(resp.raw, local_file)
        # Remove the image url response object.
        del resp
        return True

In [49]:
'''
Get school ranking, this will download the Top 100 schools, 25 schools per page, 4 pages in total
'''

dict = {}
ranking=1
for pages in range(1,5):
    if pages== 1:
        html ='https://www.niche.com/k12/search/best-private-high-schools/'
    else:
        html ='https://www.niche.com/k12/search/best-private-high-schools/?page='+ str(pages) 
    page=get_response(html)
    time.sleep(2)
    soup = BeautifulSoup(page) 
    schools = soup.find_all(match_class(["search-result__title"]))
    for school in schools:
        dict[ranking] = school.get_text()
        ranking+=1
    print("Finished with page No. "+ str(pages) +"/ Total 4 pages")
school_ranking = pd.DataFrame.from_dict(dict, orient='index')
school_ranking.to_csv("School Ranking",header = False)

Finished with page No. 1/ Total 4 pages
Finished with page No. 2/ Total 4 pages
Finished with page No. 3/ Total 4 pages
Finished with page No. 4/ Total 4 pages


In [ ]:
'''
Web-scrapping a whole webpage, for example a wiki page, not necessarily used in this project
'''
def filter_img(soup):
    links = soup.findAll('img')
    if len(links)==0:
        raise Exception('No matched tag')
    else:
        return links
    
def get_pictures(url):
    page = get_response(url)
    soup = BeautifulSoup(page)
    links = filter_img(soup)
    img_urls={}
    for i in range(0,len(links)):
        if links[i].get('src').startswith('http'):
            img_urls[i] = links[i].get('src')
    src = pd.DataFrame.from_dict(img_urls,orient='index',columns=['urls']).drop_duplicates()
    src.to_csv("testdf")
    for index in range(1,src.size):
        img_url=src.iloc[index]['urls']
        # Open the url image, set stream to True, this will return the stream content.
        resp = requests.get(img_url, stream=True)
        # Open a local file with wb ( write binary ) permission.
        local_file = open('./images/local_image'+str(index)+'.jpg', 'wb')
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        resp.raw.decode_content = True
        # Copy the response stream raw data to local image file.
        shutil.copyfileobj(resp.raw, local_file)
        # Remove the image url response object.
        del resp
    return True